<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-10 16:09:05
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: 62.54 K (0.44%)
Current PnL: -28.22 L (-18.0%)
CY Booked + Current PnL: -13.48 L (-8.6%)
-------------------
Total profit:  1.70 L
Total loss:  -29.93 L
-------------------
Total Booked + Current PnL: 13.50 L (10.4%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.43%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.61 L (66.26%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,1.08,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,62.0,H-SC,3.65,168.0,0.15,0.71,46.92,XR,ATH,FINANCE
18,COALINDIA,-0.17,5.98,12.50,19.23,19932.0,8992.0,159452.0,484.83,24.27,54.0,L-LC,12.32,182.0,0.45,1.13,28.52,XY25,ATH,MINING
77,TTKPRESTIG,1.21,-23.96,31.64,0.09,24245.0,-24149.0,76628.0,770.00,77.80,49.0,M-SC,8.39,253.0,-1.00,0.54,5.46,OX40N,NTT,DURABLES
5,ANGELONE,2.49,13.96,8.98,24.20,31239.0,42625.0,347875.0,3033.00,62.46,62.0,X-SC,2.41,99.0,1.36,2.46,45.70,X40N,NTT,FINANCE
36,ICICIGI,1.13,1.15,17.98,19.34,36395.0,2309.0,202418.0,2252.93,-20.26,57.0,X-MC,3.65,71.0,0.06,1.43,17.28,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JCHAC,7.65,-37.76,60.65,-0.01,48246.0,-48257.0,79548.0,2282.00,16182.61,54.0,M-SC,17.08,235.0,-1.00,0.56,10.29,OX40N,BTT,AC
7,ATULAUTO,6.82,-10.63,62.89,45.57,110794.0,-20961.0,176171.0,844.00,3824.32,77.0,M-SC,7.87,245.0,-0.19,1.25,33.32,XY24,NTT,AUTO
14,BSOFT,4.56,-15.61,78.67,50.79,93383.0,-21951.0,118702.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.24,0.84,38.21,XR,ATH,IT
26,FINCABLES,4.32,4.73,98.52,107.91,152341.0,6986.0,154630.0,1641.55,-14.88,71.0,M-SC,10.88,220.0,0.05,1.09,16.33,OX40N,ATH,CABLES
28,GLAXO,4.24,5.29,32.15,39.15,67461.0,10548.0,209832.0,3466.20,-15.47,67.0,X-MC,14.00,62.0,0.16,1.49,32.93,X40,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,-3.91,-6.43,44.95,35.63,88144.0,-13481.0,196093.0,1286.17,-19.84,46.0,H-MC,6.16,119.0,-0.15,1.39,9.17,AR,ATH,PHARMA
79,VAIBHAVGBL,-3.86,-23.17,109.25,60.77,153414.0,-42350.0,140425.0,521.00,61.98,59.0,H-SC,3.13,158.0,-0.28,0.99,27.31,XR,NTT,JEWELLERY
76,TRIDENT,-2.97,-26.40,74.61,28.51,50660.0,-24354.0,67900.0,48.00,-12.81,54.0,M-SC,9.13,226.0,-0.48,0.48,16.19,XR,NTT,TEXTILES
83,WHIRLPOOL,-2.59,-29.20,156.96,81.92,127566.0,-33525.0,81273.0,2270.00,-58.54,60.0,M-SC,32.30,218.0,-0.26,0.58,14.16,XR,NTT,DURABLES
51,MEDANTA,-1.86,-4.87,30.43,24.08,36749.0,-6184.0,120766.0,1486.00,-12.55,52.0,X-SC,11.48,91.0,-0.17,0.86,10.61,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,4.32,4.73,98.52,107.91,152341.0,6986.0,154630.0,1641.55,-14.88,71.0,M-SC,10.88,220.0,0.05,1.09,16.33,OX40N,ATH,CABLES
66,SIS,0.80,-20.00,51.57,21.26,45809.0,-22203.0,88829.0,528.00,2101.04,68.0,H-SC,4.38,163.0,-0.48,0.63,22.46,OX40N,NTT,MISC
72,TATAELXSI,2.83,-22.45,70.25,32.03,71823.0,-29596.0,102239.0,9161.00,-16.04,49.0,H-SC,6.20,157.0,-0.41,0.72,14.52,OX40N,NTT,IT
77,TTKPRESTIG,1.21,-23.96,31.64,0.09,24245.0,-24149.0,76628.0,770.00,77.80,49.0,M-SC,8.39,253.0,-1.00,0.54,5.46,OX40N,NTT,DURABLES
70,SYMPHONY,0.31,-26.69,36.36,-0.03,45619.0,-45676.0,125465.0,1306.00,-41.14,66.0,M-SC,11.09,206.0,-1.00,0.89,15.94,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.17,5.98,12.50,19.23,19932.0,8992.0,159452.0,484.83,24.27,54.0,L-LC,12.32,182.0,0.45,1.13,28.52,XY25,ATH,MINING
50,MASFIN,1.08,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,62.0,H-SC,3.65,168.0,0.15,0.71,46.92,XR,ATH,FINANCE
26,FINCABLES,4.32,4.73,98.52,107.91,152341.0,6986.0,154630.0,1641.55,-14.88,71.0,M-SC,10.88,220.0,0.05,1.09,16.33,OX40N,ATH,CABLES
1,ABB,0.94,7.70,35.16,45.57,99067.0,20141.0,281760.0,7934.00,-30.81,74.0,H-MC,3.59,121.0,0.20,1.99,25.12,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.94,7.70,35.16,45.57,99067.0,20141.0,281760.0,7934.00,-30.81,74.0,H-MC,3.59,121.0,0.20,1.99,25.12,AR,NTT,ELECTRICAL
50,MASFIN,1.08,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,62.0,H-SC,3.65,168.0,0.15,0.71,46.92,XR,ATH,FINANCE
26,FINCABLES,4.32,4.73,98.52,107.91,152341.0,6986.0,154630.0,1641.55,-14.88,71.0,M-SC,10.88,220.0,0.05,1.09,16.33,OX40N,ATH,CABLES
39,INDIAMART,0.12,-3.27,113.72,106.72,135666.0,-4038.0,119298.0,4810.62,-52.15,54.0,H-SC,7.91,138.0,-0.03,0.84,19.34,AR,ATH,MISC
84,WIPRO,0.60,-5.06,81.45,72.27,131030.0,-8576.0,160872.0,420.00,-18.33,38.0,M-LC,3.98,101.0,-0.07,1.14,3.40,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.82,-18.29,48.26,21.14,89644.0,-41586.0,185753.0,680.00,-24.04,26.0,X-MC,9.46,75.0,-0.46,1.32,0.00,XY24,NTT,PAINTS
6,ASIANPAINT,-0.98,-16.34,44.56,20.95,93860.0,-41131.0,210637.0,3460.25,-17.22,28.0,X-LC,8.45,36.0,-0.44,1.49,13.48,X40,ATH,PAINTS
42,INFY,0.04,1.96,31.66,34.24,101005.0,6143.0,319031.0,1972.00,-20.67,30.0,X-LC,5.34,6.0,0.06,2.26,10.41,X40,NTT,IT
8,AWL,-0.66,-31.99,131.12,57.19,290839.0,-104316.0,221811.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.09,XY24,NTT,FMCG
33,HCLTECH,-1.80,2.09,21.30,23.83,52606.0,5053.0,246977.0,1908.19,6.40,34.0,X-LC,6.85,13.0,0.10,1.75,18.49,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.23,-17.94,44.46,18.55,138003.0,-67839.0,310398.0,4311.59,-29.45,39.0,X-LC,1.19,3.0,-0.49,2.20,5.60,X40,ATH,IT
81,VBL,-0.19,-7.79,47.13,35.67,137265.0,-24595.0,291247.0,671.64,-18.74,42.0,X-LC,1.27,5.0,-0.18,2.06,4.99,X40N,ATH,FMCG
21,DABUR,1.03,1.61,41.43,43.70,104426.0,3987.0,252055.0,735.00,-6.68,58.0,X-MC,1.95,73.0,0.04,1.78,16.90,XY24,BTT,FMCG
5,ANGELONE,2.49,13.96,8.98,24.20,31239.0,42625.0,347875.0,3033.00,62.46,62.0,X-SC,2.41,99.0,1.36,2.46,45.70,X40N,NTT,FINANCE
34,HINDUNILVR,0.76,-4.55,19.09,13.67,46839.0,-11703.0,245360.0,2922.00,-11.77,63.0,X-LC,2.47,9.0,-0.25,1.74,15.83,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.82,-18.29,48.26,21.14,89644.0,-41586.0,185753.0,680.00,-24.04,26.0,X-MC,9.46,75.0,-0.46,1.32,0.00,XY24,NTT,PAINTS
8,AWL,-0.66,-31.99,131.12,57.19,290839.0,-104316.0,221811.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.09,XY24,NTT,FMCG
2,ABBOTINDIA,-0.67,-8.43,28.94,18.07,47395.0,-15085.0,163770.0,35195.00,-21.66,45.0,X-MC,9.63,64.0,-0.32,1.16,2.48,X40,ATH,PHARMA
53,PGHH,0.55,-10.26,47.10,32.01,90251.0,-21904.0,191616.0,17616.87,-36.94,47.0,X-MC,8.91,60.0,-0.24,1.36,3.57,X40,ATH,FMCG
3,ACC,-0.63,-28.75,130.39,64.15,221063.0,-68411.0,169540.0,3906.00,-58.50,48.0,X-SC,7.65,84.0,-0.31,1.20,3.63,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,2.85,-26.86,95.31,42.84,45313.0,-17463.0,47543.0,424.00,-52.07,63.0,X-SC,20.34,83.0,-0.39,0.34,14.36,XY24,NTT,MISC
58,RELAXO,0.59,-51.66,215.07,52.31,150917.0,-74989.0,70171.0,1176.00,-48.50,44.0,X-SC,13.06,92.0,-0.50,0.50,4.35,X40N,NTT,FOOTWEAR
12,BATAINDIA,3.89,-40.75,128.19,35.20,97732.0,-52430.0,76240.0,2096.00,-0.07,60.0,X-SC,21.05,93.0,-0.54,0.54,8.35,X40,NTT,FOOTWEAR
51,MEDANTA,-1.86,-4.87,30.43,24.08,36749.0,-6184.0,120766.0,1486.00,-12.55,52.0,X-SC,11.48,91.0,-0.17,0.86,10.61,XY24,NTT,HEALTHCARE
35,HONAUT,-0.94,-20.96,81.32,43.32,104691.0,-34132.0,128740.0,58357.33,-32.13,44.0,X-SC,9.75,90.0,-0.33,0.91,4.53,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.43,-21.23,40.63,10.78,76915.0,-51013.0,189305.0,452.00,-53.00,42.0,X-LC,18.73,1.0,-0.66,1.34,4.27,X40,NTT,FMCG
75,TMPV,0.52,-22.25,58.17,22.98,102169.0,-50259.0,175639.0,600.00,-19.41,66.0,X-LC,3.92,2.0,-0.49,1.24,12.28,XY24,NTT,AUTO
73,TCS,1.23,-17.94,44.46,18.55,138003.0,-67839.0,310398.0,4311.59,-29.45,39.0,X-LC,1.19,3.0,-0.49,2.20,5.60,X40,ATH,IT
81,VBL,-0.19,-7.79,47.13,35.67,137265.0,-24595.0,291247.0,671.64,-18.74,42.0,X-LC,1.27,5.0,-0.18,2.06,4.99,X40N,ATH,FMCG
42,INFY,0.04,1.96,31.66,34.24,101005.0,6143.0,319031.0,1972.00,-20.67,30.0,X-LC,5.34,6.0,0.06,2.26,10.41,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.03,-31.52,94.05,32.89,49727.0,-24334.0,52873.0,1800.00,-752.71,59.0,L-MC,11.49,259.0,-0.49,0.37,45.61,XR,NTT,BANKS
67,SONACOMS,0.09,-8.12,51.39,39.09,47763.0,-8217.0,92942.0,804.02,-28.09,67.0,M-SC,2.31,227.0,-0.17,0.66,31.48,AR,ATH,AUTO
50,MASFIN,1.08,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,62.0,H-SC,3.65,168.0,0.15,0.71,46.92,XR,ATH,FINANCE
14,BSOFT,4.56,-15.61,78.67,50.79,93383.0,-21951.0,118702.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.24,0.84,38.21,XR,ATH,IT
79,VAIBHAVGBL,-3.86,-23.17,109.25,60.77,153414.0,-42350.0,140425.0,521.00,61.98,59.0,H-SC,3.13,158.0,-0.28,0.99,27.31,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,2.49,13.96,8.98,24.20,31239.0,42625.0,347875.0,3033.00,62.46,62.0,X-SC,2.41,99.0,1.36,2.46,45.70,X40N,NTT,FINANCE
7,ATULAUTO,6.82,-10.63,62.89,45.57,110794.0,-20961.0,176171.0,844.00,3824.32,77.0,M-SC,7.87,245.0,-0.19,1.25,33.32,XY24,NTT,AUTO
67,SONACOMS,0.09,-8.12,51.39,39.09,47763.0,-8217.0,92942.0,804.02,-28.09,67.0,M-SC,2.31,227.0,-0.17,0.66,31.48,AR,ATH,AUTO
1,ABB,0.94,7.70,35.16,45.57,99067.0,20141.0,281760.0,7934.00,-30.81,74.0,H-MC,3.59,121.0,0.20,1.99,25.12,AR,NTT,ELECTRICAL
11,BANDHANBNK,2.85,-19.81,139.97,92.42,312644.0,-55194.0,223365.0,400.00,57.33,72.0,H-SC,8.80,169.0,-0.18,1.58,29.09,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.33
1,25,44.53
2,50,76.38


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.90
MC    30.64
LC    23.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.66
X40      23.21
X40N     15.18
XR        8.91
AR        8.89
XY25      8.71
OX40N     7.55
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.03
H-SC    22.98
X-LC    20.09
M-SC    11.71
X-SC    10.42
H-MC     4.85
M-MC     1.39
M-LC     1.14
L-LC     1.13
H-LC     1.10
L-SC     0.79
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.97,-12.01,49.65
FINANCE,13.45,-9.93,53.18
IT,12.17,-24.43,86.58
MISC,7.15,-28.64,81.27
ELECTRICAL,6.04,-9.85,49.93
PAINTS,5.07,-29.14,48.32
INSURANCE,4.72,-1.68,36.48
PHARMA,4.04,-3.16,35.63
AUTO,3.15,-17.86,58.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3207996.0,21
AR,1315017.0,10
XR,1231223.0,12
X40,1219064.0,15
X40N,978337.0,10
OX40N,723990.0,10
XY25,391490.0,6
SR,293999.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3545438.0,24
X-MC,1635077.0,16
M-SC,1405526.0,15
X-LC,1053439.0,12
X-SC,821493.0,9
H-MC,393513.0,3
L-SC,178329.0,2
M-LC,131030.0,1
H-LC,78005.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221109.0      6
           AR         929574.0      5
M-SC       XY24       789928.0      6
H-SC       XR         781193.0      7
X-MC       X40        536295.0      7
X-LC       X40        519693.0      6
X-MC       XY24       484909.0      3
           X40N       402430.0      4
X-SC       X40N       355292.0      4
H-SC       OX40N      319563.0      4
M-SC       OX40N      317145.0      5
X-SC       XY24       303125.0      3
H-SC       SR         293999.0      2
X-LC       X40N       220615.0      2
X-MC       XY25       211443.0      2
H-MC       XY24       206302.0      1
X-LC       XY24       202623.0      2
H-MC       AR         187211.0      2
M-SC       XR         178226.0      2
X-SC       X40        163076.0      2
M-LC       XR         131030.0      1
M-SC       AR         120227.0      2
X-LC       XY25       110508.0      2
L-SC       XR          91047.0      1
           OX40N       87282.0      1
H-LC       AR          78005.0      1
L-MC       XR          49727.0      1
M-MC       XY25        49607.0      1
L-LC       XY25        19932.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
